In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import pandas as pd

# Load environment variables from .env
load_dotenv()
os.chdir("..")
# Get API key from environment variables
OPENAI_API_KEY = os.getenv('OPEN_API_KEY')
client = OpenAI(api_key = OPENAI_API_KEY)
print(OPENAI_API_KEY)

None


## Fine-tuning improves on few-shot learning by training on many more examples than can fit in the prompt


Prepare and upload training data \
Train a new fine-tuned model \
Evaluate results and go back to step 1 if needed \
Use your fine-tuned model

### Prepare and upload training data ( create a diverse set of demonstration conversations that are similar to the conversations you will ask the model to respond to at inference time in production )


In [ ]:
import pandas as pd
import os

# Load the dataset
data_path = 'datasets'
df = pd.read_csv(os.path.join(data_path, 'wine_quality_cleaned.csv'))
df = df.drop(columns=['Unnamed: 0'])
df['description_length'] = df['description'].str.len()
# Display basic information about the dataset
df.head()

,country,description,points,price,variety,description_length
0,US,good dry creek zin robust dry spicy really get...,89,25.0,Zinfandel,175
1,France,herbaceous character make wine seem rather thi...,84,20.0,Bordeaux-style White Blend,115
2,US,little simple easy wealth raspberry strawberry...,84,19.0,Rosé,110
3,US,dry farmed vineyard treated wild yeast minimal...,88,38.0,Petite Sirah,172
4,US,site near annapolis wine show preponderance da...,91,62.0,Pinot Noir,169


In [3]:
import pandas as pd
import json

# Define the system message
system_message = {
    "role": "system",
    "content": "You are a wine classification assistant that predicts the country of origin based on wine features."
}

# Convert DataFrame to JSONL format
jsonl_data = []
for _, row in df.iterrows():
    entry = {
        "messages": [
            system_message,  # The system instruction
            {"role": "user", "content": f"Price: {row['price']}, Score: {row['points']}, Review: '{row['description']}'"},
            {"role": "assistant", "content": row["country"]}  # The correct country classification
        ]
    }
    jsonl_data.append(entry)

# Save as JSONL file
jsonl_file_path = "wine_classification.jsonl"
with open(jsonl_file_path, "w") as f:
    for entry in jsonl_data:
        f.write(json.dumps(entry) + "\n")

print(f"JSONL file saved: {jsonl_file_path}")


JSONL file saved: wine_classification.jsonl


test/ train split

In [4]:
import json
import random

# Load JSONL data from file
jsonl_file_path = "wine_classification.jsonl"
with open(jsonl_file_path, "r") as f:
    jsonl_data = [json.loads(line) for line in f]

# Shuffle the data to ensure random splitting
random.shuffle(jsonl_data)

# Define split ratio (e.g., 80% train, 20% test)
train_ratio = 0.8
split_index = int(len(jsonl_data) * train_ratio)

# Split into training and testing sets
train_data = jsonl_data[:split_index]
test_data = jsonl_data[split_index:]

# Save training data
train_file_path = "wine_train.jsonl"
with open(train_file_path, "w") as f:
    for entry in train_data:
        f.write(json.dumps(entry) + "\n")

# Save testing data
test_file_path = "wine_test.jsonl"
with open(test_file_path, "w") as f:
    for entry in test_data:
        f.write(json.dumps(entry) + "\n")

# Provide confirmation
{"message": f"Train file saved: {train_file_path}, Test file saved: {test_file_path}"}


{'message': 'Train file saved: wine_train.jsonl, Test file saved: wine_test.jsonl'}

# Some checks

In [5]:
import json

data_path = "wine_test.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 200
First example:
{'role': 'system', 'content': 'You are a wine classification assistant that predicts the country of origin based on wine features.'}
{'role': 'user', 'content': "Price: 15.0, Score: 84, Review: 'absolutely easy drink soft silky wine pleasantly ripe berry cherry flavor rich asian spiciness throughout dry balanced'"}
{'role': 'assistant', 'content': 'US'}


# debugging

In [6]:
from collections import defaultdict

# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        function_call = message.get("function_call", None)

        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


# token counts

In [7]:
import tiktoken
import numpy as np
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

# useful features

In [8]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 16385 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 16,385 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 53, 131
mean / median: 80.53, 79.0
p5 / p95: 68.0, 96.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 1, 1
mean / median: 1.0, 1.0
p5 / p95: 1.0, 1.0

0 examples may be over the 16,385 token limit, they will be truncated during fine-tuning


# cost estimation

In [9]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 16385

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

Dataset has ~16106 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~48318 tokens


uploading file

In [10]:
from openai import OpenAI

client = OpenAI()

# Upload the JSONL file
file_upload = client.files.create(
    file=open("wine_train.jsonl", "rb"), purpose="fine-tune"
)

file_id = file_upload.id  # Store the file ID
print(f"Uploaded file ID: {file_id}")


Uploaded file ID: file-9xLC8TWVGAhWhiDw2raUUw


In [11]:
job = client.fine_tuning.jobs.create(
    training_file=file_id,
    model="gpt-4o-mini-2024-07-18",
    method={
        "type": "supervised",
        "supervised": {
            "hyperparameters": {
                "batch_size": 25,
                "learning_rate_multiplier": 1.8,
                "n_epochs": 5
            }
        }
    }
)

Use a fine-tuned model

In [12]:
from openai import OpenAI
client = OpenAI()

fine_tuned_model="ft:gpt-4o-mini-2024-07-18:personal::Ay5wZDtW"
fine_tuned_model_j="ft:gpt-4o-mini-2024-07-18:personal::AyGh2lZU"

wine_features = "Price: 30, Score: 90, Review: 'Rich and full-bodied with dark fruit notes and a smooth finish.'"

completion = client.chat.completions.create(
    model=fine_tuned_model,
    messages=[
        {"role": "system", "content": "You are a wine classification assistant that predicts the country of origin based on wine features."},
        {"role": "user", "content": wine_features}
    ]
)

# Extract the predicted country
predicted_country = completion.choices[0].message.content

print(f"Predicted Country of Origin: {predicted_country}")

Predicted Country of Origin: US


Analyzing your fine-tuned model

In [13]:
fj_job = 'ftjob-ljho4V9A6Kb8Wsmw8ukeAoZJ'

fj_job_j = 'ftjob-1lAVUjpk5pirPK05680s22qA'

job_details = client.fine_tuning.jobs.retrieve(fj_job)
job_details_j = client.fine_tuning.jobs.retrieve(fj_job_j)

In [14]:
import json

# Load test data
test_file_path = "wine_test.jsonl"

with open(test_file_path, "r") as f:
    test_data = [json.loads(line) for line in f]

In [15]:
correct_predictions = 0
total_predictions = len(test_data)

for entry in test_data:
    user_input = entry["messages"][1]["content"]  # Extract wine features
    actual_country = entry["messages"][2]["content"]  # Correct label

    # Query fine-tuned model for prediction
    completion = client.chat.completions.create(
        model="ft:gpt-4o-mini-2024-07-18:personal::Ay5wZDtW",  # Your fine-tuned model
        messages=[
            {"role": "system", "content": "You are a wine classification assistant that predicts the country of origin based on wine features."},
            {"role": "user", "content": user_input}
        ]
    )

    # Extract predicted country
    predicted_country = completion.choices[0].message.content.strip()

    # Compare prediction with actual label
    if predicted_country.lower() == actual_country.lower():
        correct_predictions += 1

# Compute accuracy
accuracy = (correct_predictions / total_predictions) * 100

# Print results
print(f"Total Test Samples: {total_predictions}")
print(f"Correct Predictions: {correct_predictions}")
print(f"Model Accuracy: {accuracy:.2f}%")


KeyboardInterrupt: 

In [ ]:
correct_predictions = 0
total_predictions = len(test_data)

for entry in test_data:
    user_input = entry["messages"][1]["content"]  # Extract wine features
    actual_country = entry["messages"][2]["content"]  # Correct label

    # Query fine-tuned model for prediction
    completion = client.chat.completions.create(
        model=fine_tuned_model_j,  # Your fine-tuned model
        messages=[
            {"role": "system", "content": "You are a wine classification assistant that predicts the country of origin based on wine features."},
            {"role": "user", "content": user_input}
        ]
    )

    # Extract predicted country
    predicted_country = completion.choices[0].message.content.strip()

    # Compare prediction with actual label
    if predicted_country.lower() == actual_country.lower():
        correct_predictions += 1

# Compute accuracy
accuracy = (correct_predictions / total_predictions) * 100

# Print results
print(f"Total Test Samples: {total_predictions}")
print(f"Correct Predictions: {correct_predictions}")
print(f"Model Accuracy: {accuracy:.2f}%")

Total Test Samples: 200
Correct Predictions: 182
Model Accuracy: 91.00%


In [ ]:
correct_predictions = 0
total_predictions = len(test_data)

for entry in test_data:
    user_input = entry["messages"][1]["content"]  # Extract wine features
    actual_country = entry["messages"][2]["content"]  # Correct label

    # Query fine-tuned model for prediction
    completion = client.chat.completions.create(
        model=fine_tuned_model_j,  # Your fine-tuned model
        messages=[
            {"role": "system", "content": "You are a wine classification assistant that predicts the country of origin based on wine features."},
            {"role": "user", "content": user_input}
        ]
    )

    # Extract predicted country
    predicted_country = completion.choices[0].message.content.strip()

    # Compare prediction with actual label
    if predicted_country.lower() == actual_country.lower():
        correct_predictions += 1

# Compute accuracy
accuracy = (correct_predictions / total_predictions) * 100

# Print results
print(f"Total Test Samples: {total_predictions}")
print(f"Correct Predictions: {correct_predictions}")
print(f"Model Accuracy: {accuracy:.2f}%")

Total Test Samples: 200
Correct Predictions: 180
Model Accuracy: 90.00%


In [ ]:
import boto3

sm = boto3.client("sagemaker")

response = sm.create_model(
    ModelName="WineModel",
    PrimaryContainer={
        "Image": "123456789012.dkr.ecr.us-east-1.amazonaws.com/scikit-learn-inference",
        "ModelDataUrl": "s3://your-bucket-name/wine_model.pkl"
    },
    ExecutionRoleArn="arn:aws:iam::123456789012:role/SageMakerRole"
)


ClientError: An error occurred (ValidationException) when calling the CreateModel operation: RoleArn: Cross-account pass role is not allowed.

In [ ]:
add#


NameError: name 'add' is not defined